In [1]:
import cv2
import math
import random
import numpy as np

In [2]:
def drawText(image, txt):
    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (50, 50)
    fontScale = 1
    color = (255, 0, 0) # in BGR
    thickness = 2
    image = cv2.putText(image, txt, org, font, fontScale, color, thickness, cv2. LINE_AA)

In [3]:
def distance(p1, p2):
    dist = math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)
    return dist

In [4]:
def sort_contours(cnts, method = 'left_to_right'):
    reverse = False
    i = 0
    if method == 'right_to_left' or method == 'bottom_to_top':
        reverse = True
    
    # handle the y-coordinate
    if method == 'top_to_bottom' or method == 'bottom_to_top':
        i = 1
    
    # construct the list of bounding boxes and sort them from top to bottom
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    
    cnts = [cnts for cnts, boundingBoxes in sorted(zip(cnts, boundingBoxes), key = lambda b:b[1][i], reverse = reverse)]
    
    return cnts

In [5]:
def order_points(pts):
    rect = np.zeros((4,2), dtype = 'float32')
    
    s = np.sum(pts, axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    
    return rect

In [6]:
def four_points_transform(image, pts):
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    width = max(int(widthA), int(widthB))
    
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    height = max(int(heightA), int(heightB))
    dst = np.array([[0,0],
                   [width-1, 0],
                   [width-1, height-1],
                   [0, height-1]], dtype = 'float32')
    
    m = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, m, (width, height))
    
    return warped

In [7]:
def find_corner_by_rotated_rect(box, approx):
    corner = []
    for p_box in box:
        min_dist = 999999999
        min_p = None
        for p in approx:
            dist = distance(p_box, p[0])
            if dist < min_dist:
                min_dist = dist
                min_p = p[0]
        corner.append(min_p)
    corner = np.array(corner)
    return corner

In [8]:
# 1. Load image
image = cv2.imread('test.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

In [9]:
# 2. Threshold
thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 31, 3)
cv2.imshow('2. Threshold', thresh)
cv2.imwrite('Step 2.jpg', thresh)
cv2.waitKey()

-1

In [10]:
# 3. Find the outer frame
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=lambda x: cv2.contourArea(x),reverse=True)
approx = cv2.approxPolyDP(contours[1], 0.01 * cv2.arcLength(contours[1], True), True)
rect = cv2.minAreaRect(contours[1])
box = cv2.boxPoints(rect)

In [11]:
# 4. Realize transform
corner = find_corner_by_rotated_rect(box,approx)
image = four_points_transform(image,corner)
wrap = four_points_transform(thresh,corner)
cv2.imshow('After step 4', wrap)
cv2.imwrite('Step 4.jpg', wrap)
cv2.waitKey()

-1

In [12]:
# 5. Find the ticks
contours, _ = cv2.findContours(wrap, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
tickcontours = []

for c in contours:
    (x, y, w, h) = cv2.boundingRect(c)
    ar = w/float(h)
    if w >= 30 and h >= 30 and 0.8 <= ar <= 1.2:
        tickcontours.append(c)

In [13]:
# 6. Compare with answers
right_answers = {0:1, 1:4, 2:0, 3:3, 4:1, 5:1, 6:0, 7:4}

tickcontours = sort_contours(tickcontours, method='top_to_bottom')

correct = 0

for (q, i) in enumerate(np.arange(0, len(tickcontours), 5)):
    color = (random.randint(0, 255), random.randint(0, 255), random.randint(0 , 255))
    cnts = sort_contours(tickcontours[i:i + 5])
    #cv2.drawContours(image, cnts, -1, color, 3)
    choice = (0,0)
    total = 0
    
    for (j, c) in enumerate(cnts):
        # Create a mask to see the colored level of the contour
        mask = np.zeros(wrap.shape, dtype="uint8")
        cv2.drawContours(mask, [c], -1, 255, -1)
        mask = cv2.bitwise_and(wrap, wrap, mask=mask)
        total = cv2.countNonZero(mask)

        if total > choice[0]:
            choice = (total, j)

    current_right = right_answers[q]
 
    if current_right == choice[1]:
        color = (0, 255, 0)
        correct += 1
    else:
        color = (0, 0, 255)
    cv2.drawContours(image, [cnts[current_right]], -1, color, 3)

    
drawText(image, 'Correct:' + str(correct))

cv2.imshow('Final', image)
cv2.imwrite('Final.jpg', image)
cv2.waitKey()

-1